# Проверка статистических гипотез

In [17]:
import numpy as np
from scipy import stats
import statsmodels.api as sm

## Реализация

### Тест для пропорций

In [18]:
def props_test(m, m_cases, n, n_cases, alternative='two_sided'):
    p_1 = m / m_cases
    p_2 = n / n_cases
    p = (m + n) / (m_cases + n_cases)
    z = (p_1 - p_2) / np.sqrt(p*(1-p)*(1/m_cases+1/n_cases))
    cdf = stats.norm.cdf(z)
    if alternative == 'greater':
        return 1 - cdf
    if alternative == 'less':
        return cdf
    return np.min([2*cdf, 2 - 2*cdf])

### Критерий Манна - Уитни

In [19]:
def get_ranks(x, y):
    merged = np.hstack([x, y])
    merged = np.sort(merged)
    _, counts = np.unique(merged, return_counts=True)
    a1 = np.cumsum(counts) - counts
    a2 = np.cumsum(counts)
    s = a2*(a2+1)/2 - (a1)*(a1+1)/2
    ranks = np.repeat(s / counts, counts)
    return merged, ranks


def u_test(x, y, alternative='two_sided'):
    x, y = np.array(x), np.array(y)
    m, n = x.shape[0], y.shape[0]
    merged, ranks = get_ranks(x, y)
    y_indices = np.argmax(merged == y.reshape(-1, 1), axis=1)
    y_ranks = ranks[y_indices]
    u_y = np.sum(y_ranks)
    exp = n/2*(m+n+1)
    var = m*n/12*(m+n+1)
    z = (u_y - exp) / np.sqrt(var)
    cdf = stats.norm.cdf(z)
    if alternative == 'greater':
        # среднее 1ой выборки больше
        return cdf
    if alternative == 'less':
        # среднее 2ой выборки больше
        return 1 - cdf
    return np.min([2*cdf, 2 - 2*cdf])

## Задача 1

Перед президентскими выборами в городах Курске и Владивостоке был проведен
социологический опрос. Каждый респондент должен был ответить на вопрос: «За какого
кандидата вы будете голосовать на выборах?». В Курске опросили 105 человек, из них 42
ответили, что будут голосовать за кандидата А, во Владивостоке опросили 195 человек, из которых
65 за А. Можно ли считать на уровне значимости 0,05, что уровни поддержки кандидата А в Курске
и Владивостоке одинаковы?

$H_0$: $p_1=p_2$ - уровни поддержки кандидата А в Курске и Владивостоке одинаковы

$H_A$: $p_1 \gt p_2$ - уровень поддержки кандидата А в Курске выше, чем во Владивостоке

In [20]:
m = 42
M = 105
n = 65
N = 195

In [21]:
props_test(m, M, n, N, alternative='greater'), sm.stats.proportions_ztest([m, n], [M, N], alternative='larger')[1]

(0.1251194122753233, 0.1251194122753233)

Т.к. p-value $\gt \alpha$, то на заданном уровне значимости нельзя отвергнуть нулевую гипотезу об одинаковом уровне поддержки в двух городах.

## Задача 2

Для изучения эффективности лекарства против аллергии обследовались две группы людей,
предрасположенных к этому заболеванию. Результаты обследования следующие: среди
принимавших лекарство заболело 3 человека, не заболело 172 человека; среди не принимавших
заболело 32 человека, не заболело 168. Указывают ли эти результаты на эффективность
лекарства?

$H_0$: $p_1=p_2$ - процент заболевших среди принимавших и среди не принимавших лекарство одинаков

$H_A$: $p_1 \lt p_2$ - процент заболевших среди принимавших лекарство меньше (лекарство эффективно)

Уровень значимости: $\alpha = 0.05$

In [9]:
m = 3
M = 172 + m
n = 32
N = 168 + n

In [10]:
props_test(m, M, n, N, alternative='less'), sm.stats.proportions_ztest([m, n], [M, N], alternative='smaller')[1]

(1.045804697525518e-06, 1.045804697525518e-06)

Т.к. p-value $\lt \alpha$, то на заданном уровне значимости можно отвергнуть нулевую гипотезу в пользу альтернативной.

## Задача 3

Было проведено обследование 10 горожан и 9 жителей села примерно одного возраста.
Получены следующие данные об уровне давления:

- для горожан:

In [11]:
x = [130, 110, 120, 140, 200, 130, 140, 170, 160, 140]

- для селян:

In [12]:
y = [120, 190, 130, 160, 150, 120, 110, 120, 200]

Свидетельствуют ли эти данные в пользу того, что горожане имеют в среднем более высокое
давление чем жители сельской местности?

$H_0$: давление у горожан и селян не различаются

$H_A$: давление у горожан выше

Уровень значимости: $\alpha = 0.05$

In [13]:
u_test(x, y, alternative='greater'), stats.mannwhitneyu(x, y, alternative='greater', use_continuity=False).pvalue

(0.37198573903752846, 0.37083260206079616)

Т.к. p-value $\gt \alpha$, то на заданном уровне значимости нельзя отвергнуть нулевую гипотезу.

## Задача 4

Уровень гистамина в мокроте у 7 курильщиков, склонных к аллергии, составил в мг:

In [14]:
x = [102.4, 100.0, 67.6, 65.9, 64.7, 39.6, 31.2]

У 10 курильщиков, не склонных к аллергии, составил в мг:

In [15]:
y = [48.1, 45.5, 41.7, 35.4, 29.1, 18.9, 58.3, 68.8, 71.3, 94.3]

Можно ли, основываясь на этих данных, считать с надёжностью 0,95 что уровень гистамина у
склонных и не склонных к аллергии курильщиков значимо различается?

$H_0$: уровни гистамина у склонных и не склонных к аллергии курильщиков не различаются

$H_A$: уровень гистамина у склонных к аллергии курильщиков выше

In [16]:
u_test(x, y, alternative='greater'), stats.mannwhitneyu(x, y, alternative='greater', use_continuity=False).pvalue

(0.14152564352594232, 0.14152564352594232)

Т.к. p-value $\gt \alpha$, то на заданном уровне значимости нельзя отвергнуть нулевую гипотезу.

In [31]:
np.mean(x), np.mean(y)

(67.34285714285714, 51.14)